Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [7]:
#pickle_file = 'notMNIST.pickle'
pickle_file = 'notMNIST_noDupNorOvlp.pickle'


with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (438047, 28, 28) (438047,)
Validation set (15586, 28, 28) (15586,)
Test set (13645, 28, 28) (13645,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (438047, 784) (438047, 10)
Validation set (15586, 784) (15586, 10)
Test set (13645, 784) (13645, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [10]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.176472
Training accuracy: 11.4%
Validation accuracy: 13.7%
Loss at step 100: 2.607812
Training accuracy: 68.3%
Validation accuracy: 68.0%
Loss at step 200: 2.118773
Training accuracy: 71.3%
Validation accuracy: 70.6%
Loss at step 300: 1.846766
Training accuracy: 72.5%
Validation accuracy: 71.3%
Loss at step 400: 1.660105
Training accuracy: 73.4%
Validation accuracy: 71.7%
Loss at step 500: 1.520854
Training accuracy: 74.2%
Validation accuracy: 72.0%
Loss at step 600: 1.411626
Training accuracy: 74.9%
Validation accuracy: 72.3%
Loss at step 700: 1.322945
Training accuracy: 75.5%
Validation accuracy: 72.4%
Loss at step 800: 1.248959
Training accuracy: 75.9%
Validation accuracy: 72.6%
Test accuracy: 79.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [12]:
batch_size = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [13]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.558800
Minibatch accuracy: 8.4%
Validation accuracy: 9.6%
Minibatch loss at step 500: 1.909721
Minibatch accuracy: 70.7%
Validation accuracy: 72.5%
Minibatch loss at step 1000: 1.563437
Minibatch accuracy: 73.6%
Validation accuracy: 73.9%
Minibatch loss at step 1500: 1.072572
Minibatch accuracy: 75.6%
Validation accuracy: 74.8%
Minibatch loss at step 2000: 1.169918
Minibatch accuracy: 73.4%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 0.863057
Minibatch accuracy: 80.5%
Validation accuracy: 76.0%
Minibatch loss at step 3000: 1.038281
Minibatch accuracy: 76.6%
Validation accuracy: 76.5%
Minibatch loss at step 3500: 1.069981
Minibatch accuracy: 75.6%
Validation accuracy: 76.4%
Minibatch loss at step 4000: 0.799259
Minibatch accuracy: 79.9%
Validation accuracy: 77.2%
Minibatch loss at step 4500: 0.903940
Minibatch accuracy: 78.3%
Validation accuracy: 77.3%
Minibatch loss at step 5000: 0.680465
Minibatch accuracy: 82.6%
Validation accuracy

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [11]:
num_nodes = 1024
batch_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables Layer 1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
  # Variables Layer 2
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits_2)
    
  # Predictions for validation 
  logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  relu_layer= tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
  valid_prediction = tf.nn.softmax(logits_2)
    
  # Predictions for test
  logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  relu_layer= tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
  test_prediction =  tf.nn.softmax(logits_2)

Let's run it:

In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print('Offset in iteration '+ str(step) + ' is: '+ str(offset))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Offset in iteration 0 is: 0
Minibatch loss at step 0: 267.289307
Minibatch accuracy: 16.5%
Validation accuracy: 31.7%
Offset in iteration 1 is: 1024
Offset in iteration 2 is: 2048
Offset in iteration 3 is: 3072
Offset in iteration 4 is: 4096
Offset in iteration 5 is: 5120
Offset in iteration 6 is: 6144
Offset in iteration 7 is: 7168
Offset in iteration 8 is: 8192
Offset in iteration 9 is: 9216
Offset in iteration 10 is: 10240
Offset in iteration 11 is: 11264
Offset in iteration 12 is: 12288
Offset in iteration 13 is: 13312
Offset in iteration 14 is: 14336
Offset in iteration 15 is: 15360
Offset in iteration 16 is: 16384
Offset in iteration 17 is: 17408
Offset in iteration 18 is: 18432
Offset in iteration 19 is: 19456
Offset in iteration 20 is: 20480
Offset in iteration 21 is: 21504
Offset in iteration 22 is: 22528
Offset in iteration 23 is: 23552
Offset in iteration 24 is: 24576
Offset in iteration 25 is: 25600
Offset in iteration 26 is: 26624
Offset in iteration 27 is: 276

Offset in iteration 238 is: 243712
Offset in iteration 239 is: 244736
Offset in iteration 240 is: 245760
Offset in iteration 241 is: 246784
Offset in iteration 242 is: 247808
Offset in iteration 243 is: 248832
Offset in iteration 244 is: 249856
Offset in iteration 245 is: 250880
Offset in iteration 246 is: 251904
Offset in iteration 247 is: 252928
Offset in iteration 248 is: 253952
Offset in iteration 249 is: 254976
Offset in iteration 250 is: 256000
Offset in iteration 251 is: 257024
Offset in iteration 252 is: 258048
Offset in iteration 253 is: 259072
Offset in iteration 254 is: 260096
Offset in iteration 255 is: 261120
Offset in iteration 256 is: 262144
Offset in iteration 257 is: 263168
Offset in iteration 258 is: 264192
Offset in iteration 259 is: 265216
Offset in iteration 260 is: 266240
Offset in iteration 261 is: 267264
Offset in iteration 262 is: 268288
Offset in iteration 263 is: 269312
Offset in iteration 264 is: 270336
Offset in iteration 265 is: 271360
Offset in iteration 

Offset in iteration 476 is: 50401
Offset in iteration 477 is: 51425
Offset in iteration 478 is: 52449
Offset in iteration 479 is: 53473
Offset in iteration 480 is: 54497
Offset in iteration 481 is: 55521
Offset in iteration 482 is: 56545
Offset in iteration 483 is: 57569
Offset in iteration 484 is: 58593
Offset in iteration 485 is: 59617
Offset in iteration 486 is: 60641
Offset in iteration 487 is: 61665
Offset in iteration 488 is: 62689
Offset in iteration 489 is: 63713
Offset in iteration 490 is: 64737
Offset in iteration 491 is: 65761
Offset in iteration 492 is: 66785
Offset in iteration 493 is: 67809
Offset in iteration 494 is: 68833
Offset in iteration 495 is: 69857
Offset in iteration 496 is: 70881
Offset in iteration 497 is: 71905
Offset in iteration 498 is: 72929
Offset in iteration 499 is: 73953
Offset in iteration 500 is: 74977
Minibatch loss at step 500: 10.228548
Minibatch accuracy: 79.2%
Validation accuracy: 80.1%
Offset in iteration 501 is: 76001
Offset in iteration 502 i

Offset in iteration 711 is: 291041
Offset in iteration 712 is: 292065
Offset in iteration 713 is: 293089
Offset in iteration 714 is: 294113
Offset in iteration 715 is: 295137
Offset in iteration 716 is: 296161
Offset in iteration 717 is: 297185
Offset in iteration 718 is: 298209
Offset in iteration 719 is: 299233
Offset in iteration 720 is: 300257
Offset in iteration 721 is: 301281
Offset in iteration 722 is: 302305
Offset in iteration 723 is: 303329
Offset in iteration 724 is: 304353
Offset in iteration 725 is: 305377
Offset in iteration 726 is: 306401
Offset in iteration 727 is: 307425
Offset in iteration 728 is: 308449
Offset in iteration 729 is: 309473
Offset in iteration 730 is: 310497
Offset in iteration 731 is: 311521
Offset in iteration 732 is: 312545
Offset in iteration 733 is: 313569
Offset in iteration 734 is: 314593
Offset in iteration 735 is: 315617
Offset in iteration 736 is: 316641
Offset in iteration 737 is: 317665
Offset in iteration 738 is: 318689
Offset in iteration 

Offset in iteration 951 is: 99778
Offset in iteration 952 is: 100802
Offset in iteration 953 is: 101826
Offset in iteration 954 is: 102850
Offset in iteration 955 is: 103874
Offset in iteration 956 is: 104898
Offset in iteration 957 is: 105922
Offset in iteration 958 is: 106946
Offset in iteration 959 is: 107970
Offset in iteration 960 is: 108994
Offset in iteration 961 is: 110018
Offset in iteration 962 is: 111042
Offset in iteration 963 is: 112066
Offset in iteration 964 is: 113090
Offset in iteration 965 is: 114114
Offset in iteration 966 is: 115138
Offset in iteration 967 is: 116162
Offset in iteration 968 is: 117186
Offset in iteration 969 is: 118210
Offset in iteration 970 is: 119234
Offset in iteration 971 is: 120258
Offset in iteration 972 is: 121282
Offset in iteration 973 is: 122306
Offset in iteration 974 is: 123330
Offset in iteration 975 is: 124354
Offset in iteration 976 is: 125378
Offset in iteration 977 is: 126402
Offset in iteration 978 is: 127426
Offset in iteration 9